The food price data is obtained from [Humanitarian Data Exchange](https://data.humdata.org/dataset/?dataseries_name=WFP+-+Food+Prices).

In [19]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [3]:
country = 'Nigeria'

In [4]:
topfolder = '/Users/sahoko/GitHub/Food-insecurity-GP-forecasting/'
datafolder = os.path.join(topfolder,'Data/New/FoodPrice')
os.chdir(topfolder)

In [8]:
file = f"{datafolder}/Raw/{country}/wfp_food_prices.csv"
df = pd.read_csv(file, header=0, skiprows=[1])

In [17]:
df.loc[df['category']=='cereals and tubers']['unit'].unique()

array(['KG', '100 KG', '50 KG', 'Unit', '1.3 KG', '1.4 KG', '3.4 KG',
       '3.1 KG', '100 Tubers', '2.5 KG'], dtype=object)

In [20]:
df['Datetime'] = pd.to_datetime(df['date'])

Data cleaning
* Same as Fioni 2023 
    1. Limit the data to cereal and tuber category [2018>]
    2. price (in USD) -> adjust for the unit
    3. Interpolate if missing
    4. Subset the data with 'commodity' that are available to all the adminstrative regions of the same country
    5. Normalise each subcategory time series
    6. Take the mean over all sub-categories
    7. limit time series from starting month - end month {at the end}
* Some question
    1. pricetype / priceflag should they be all treated the same? or only using a certain pricetype or price flag
        check `df_ct['priceflag'].unique()` or `df_ct['pricetype'].unique()`
    2. USD or adjusted currency (adjusted for income in USD or so)
 
* Modified [1]
    1. 1-3 the same as above
    2. interpolate (daily or weekly...)
* Modified [2]
    1. 1-3 the same as above
    2. PCA on time series?
    

1. Limit the data to cereal and tuber category

In [233]:
df_ct = df.loc[(df['category']=='cereals and tubers') & (df['Datetime']>pd.to_datetime('2018-01-01'))].copy()

2. price (in USD) -> adjust for the unit
    By running `df_ct['unit'].unique()` we get   
    ```
    ['KG', '100 KG', '50 KG', 'Unit', '1.3 KG', '1.4 KG', '3.4 KG',
       '3.1 KG', '100 Tubers', '2.5 KG']
    ```
    ---------
   * We see that `'100 Tubers'` are used for Yam or Yam (Abuja), and `'Unit'` is used for Bread. Check `df_ct.loc[df_ct['unit']=='Unit']['commodity'].unique()` for example.
   
   * We remove the data entreies where the price is measured with `100 Tubers` but keep the one with `Unit`, i.e., bread 
   
   * Note `Yam (Abuja)` is only recorded for the admin1 region  `'Oyo'`
   ---------
   * To adjust for unit, split the numerical part and the unit e.g. `'2.5'` and  `'KG'` in `'2.5 KG'` and take the former to create a nuw column `'unit_numeric'`
   * If for each date-time, admin1 region and commodity, thre are multiple entries, take the mean

In [234]:
df_ct = df_ct.loc[df_ct['unit']!='100 Tubers'].copy()

In [235]:
df_ct['unit'].replace('KG', '1 KG', inplace=True)
df_ct['unit_numeric'] = 1.0
df_ct.loc[df_ct['unit']!='Unit',['unit_numeric']] = df_ct.loc[df_ct['unit']!='Unit']['unit'].str[:-3].astype(float)

In [236]:
df_ct['usdprice_unit'] = df_ct['usdprice']/df_ct['unit_numeric']
df_ct['price_unit'] = df_ct['price']/df_ct['unit_numeric']

In [237]:
df_ct_unique = df_ct.groupby(['Datetime', 'admin1','commodity']).agg({'usdprice_unit':'mean', 'price_unit':'mean'}).reset_index()

In [238]:
df_ct_unique_temp  =  df_ct_unique.drop('price_unit', axis=1)
df_ct_unique_temp = df_ct_unique_temp.set_index(['Datetime','admin1','commodity']).unstack('commodity').reset_index()
df_ct_unique_temp.columns  = ['_'.join(col) for col in df_ct_unique_temp.columns]
df_ct_unique_temp.rename(columns={'Datetime_':'Datetime','admin1_':'admin1'}, inplace=True)

3. Interpolate if missing
    3.1 Make dataframe with Datetime (all) X admin 1

### Need to be updated from here ###

In [239]:
region_list = list(df_ct_unique['admin1'].unique())
region_number = len(region_list)
date = list(pd.date_range(df_ct_unique['Datetime'].min() - timedelta(days=30), df_ct_unique['Datetime'].max(),freq='MS') + pd.DateOffset(days=14))
date_length = len(date)
d = {'Datetime':date*region_number,
     'admin1':np.repeat(region_list, date_length)}
df1 = pd.DataFrame(data=d)
df1 = df1.sort_values(by=['admin1','Datetime'])

In [240]:
df2  =  df_ct_unique.drop('price_unit', axis=1)
df2 = df2.set_index(['Datetime','admin1','commodity']).unstack('commodity').reset_index()
df2.columns  = ['_'.join(col) for col in df2.columns]
df2.rename(columns={'Datetime_':'Datetime','admin1_':'admin1'}, inplace=True)

In [ ]:
df_merge = pd.merge(df1,df2,how="outer",on=['Datetime','admin1'])

In [261]:
d = {'Datetime':date}
df1 = pd.DataFrame(data=d)

In [262]:
df2  =  df_ct_unique.drop('price_unit', axis=1)
df2 = df2.set_index(['Datetime','admin1','commodity']).unstack(['admin1','commodity']).reset_index()
#df2.rename(columns={'Datetime_':'Datetime','admin1_':'admin1'}, inplace=True)

In [263]:
df2.columns  = ['_'.join(col) for col in df2.columns]
df2.rename(columns={'Datetime__':'Datetime'}, inplace=True)

In [264]:
df_merge = pd.merge(df1,df2,how="outer",on=['Datetime'])

In [265]:
df_merge

,Datetime,usdprice_unit_Borno_Bread,"usdprice_unit_Borno_Cassava meal (gari, yellow)",usdprice_unit_Borno_Gari (white),usdprice_unit_Borno_Maize (white),usdprice_unit_Borno_Maize (yellow),usdprice_unit_Borno_Maize flour,usdprice_unit_Borno_Millet,usdprice_unit_Borno_Rice (imported),usdprice_unit_Borno_Rice (local),...,usdprice_unit_Abia_Maize (white),usdprice_unit_Abia_Maize (yellow),usdprice_unit_Abia_Millet,usdprice_unit_Abia_Rice (imported),usdprice_unit_Abia_Rice (local),usdprice_unit_Abia_Sorghum (brown),usdprice_unit_Abia_Sorghum (white),usdprice_unit_Abia_Yam,usdprice_unit_Adamawa_Maize flour,usdprice_unit_Adamawa_Sorghum
0,2018-01-15,0.98325,0.890050,0.646649,0.254408,0.283050,0.421942,0.419927,1.097878,0.727052,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-02-15,1.00775,0.431428,0.371372,0.233333,0.280112,0.579077,0.459466,1.075919,0.809817,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-03-15,1.09580,0.366702,0.324174,0.261905,0.284734,0.781077,0.462038,1.130120,0.765250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-04-15,1.26890,0.451755,0.376773,NaN,0.341866,0.460154,0.468748,1.280603,0.897134,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-15,1.11150,0.432778,0.353806,0.283889,0.308278,0.419725,0.350880,1.049570,0.822029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2023-08-15,NaN,NaN,NaN,NaN,NaN,0.608227,0.658443,1.468749,1.120905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.58224,0.67556
68,2023-09-15,NaN,NaN,NaN,NaN,NaN,0.626188,0.643425,1.504878,1.206729,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.71430,0.64924
69,2023-10-15,NaN,NaN,NaN,NaN,NaN,0.618551,0.627842,1.566798,1.227171,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.65626,0.66464
70,2023-11-15,NaN,NaN,NaN,NaN,NaN,0.524154,0.591335,1.475406,1.123489,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.60220,0.62756


In [266]:
def interpolation(x): 
    # Let's interpolate the NaN values using a linear interpolation up to a limit of 3 months.
    x = x.loc[x.first_valid_index():x.last_valid_index()]
    x = x.interpolate(method = "linear", limit = 3)
    # Keep only the commodities that not have any nan values.
    return x.dropna(axis = 1, how = "any")

In [225]:
a = df_merge.loc[df_merge['admin1']=='Zamfara'].set_index(['Datetime','admin1'])
a = a.loc[a.first_valid_index(): a.last_valid_index()]

In [270]:
a = df_merge.set_index('Datetime')
a.loc[a.first_valid_index(): a.last_valid_index()].interpolate(method = "linear", limit = 3).dropna(axis = 1, how = "any")

,usdprice_unit_Borno_Millet,usdprice_unit_Borno_Rice (imported),usdprice_unit_Borno_Rice (local),usdprice_unit_Borno_Yam,usdprice_unit_Yobe_Millet,usdprice_unit_Yobe_Rice (imported),usdprice_unit_Yobe_Rice (local),usdprice_unit_Yobe_Yam
Datetime,,,,,,,,
2018-01-15,0.419927,1.097878,0.727052,0.442345,0.523955,1.842621,1.217199,0.360935
2018-02-15,0.459466,1.075919,0.809817,0.440106,0.532442,1.932743,1.315930,0.336388
2018-03-15,0.462038,1.130120,0.765250,0.408580,0.522423,1.884223,1.227058,0.308316
2018-04-15,0.468748,1.280603,0.897134,0.615029,0.606532,2.129812,1.492699,0.368155
2018-05-15,0.350880,1.049570,0.822029,0.857875,0.498376,1.661325,1.286703,0.309569
...,...,...,...,...,...,...,...,...
2023-08-15,0.658443,1.468749,1.120905,0.815731,0.629613,1.141640,1.072580,0.763870
2023-09-15,0.643425,1.504878,1.206729,0.842358,0.474027,1.352528,1.134647,0.816336
2023-10-15,0.627842,1.566798,1.227171,0.867813,0.472260,1.424435,1.036995,0.880314


In [224]:
a.last_valid_index()

(Timestamp('2023-12-15 00:00:00'), 'Zamfara')

In [215]:
df_merge.loc[df_merge['admin1']=='Zamfara',['usdprice_unit_Bread']]

,usdprice_unit_Bread
3432,NaN
3433,NaN
3434,NaN
3435,NaN
3436,NaN
...,...
3691,NaN
3692,NaN
3693,NaN
3694,NaN


In [204]:
df_interpolate = df_merge.apply(interpolation)
df_interpolate.head()

ValueError: Invalid fill method. Expecting pad (ffill) or backfill (bfill). Got linear

In [114]:
df_tmp = df_ct.set_index(['Datetime','admin1','commodity'])

In [31]:
st_year = '2022'
st_month = '10'
df.loc[df['Datetime']>pd.to_datetime(f"{st_year}-{st_month}-15")]['date'].unique()

In [33]:
df.loc[df['Datetime']>pd.to_datetime(f"{st_year}-{st_month}-15")]['date'].unique()

array(['2022-02-15', '2022-03-15', '2022-04-15', '2022-05-15',
       '2023-02-15', '2023-03-15', '2023-04-15', '2023-05-15',
       '2023-06-15', '2022-06-15', '2022-07-15', '2022-08-15',
       '2022-09-15', '2022-10-15', '2022-11-15', '2022-12-15',
       '2023-01-15', '2023-07-15', '2023-08-15', '2023-09-15',
       '2023-10-15', '2023-11-15', '2023-12-15'], dtype=object)